In [115]:
get_ipython().ast_node_interactivity = 'all'
import pandas as pd
import numpy
from datetime import datetime as dt

from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
get_ipython().ast_node_interactivity = 'all'
import pandas 
import numpy

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


In [116]:
df= pd.read_csv( "final_ml_data.csv")

In [117]:
df["weekly_change"] = df["open"] - df["close"]
df["up_down"] = ["up" if change >= 0 else "down" for change in df["weekly_change"]]
df['articleDate'] = pd.to_datetime(df['articleDate'])
df['percent_change'] = ((df['close']/df['open']) - 1) * 100
df['invest'] = df.apply(lambda row: 'invest' if (row['close'] / row['open'] - 1) >= 0.01 else 'no invest', axis=1)
df['day_of_week'] = df['articleDate'].dt.day_name()

# Assuming df is your DataFrame
# Define a function to check the count and replace with 'other' if count is 1
def replace_source(source):
    if df['source'].value_counts()[source] <= 50:
        return 'other'
    else:
        return source

# Apply the function to the 'source' column
df['source'] = df['source'].apply(replace_source)

In [118]:
df
df.dtypes

,articleDate,n_companies,StockSymbol,source,companies,entities,positive_sentiment,negative_sentiment,neutral_sentiment,title_polarity,...,volume_end,volume_mean,country_isUS,open_marketcap,close_marketcap,weekly_change,up_down,percent_change,invest,day_of_week
0,2022-01-03,2,AAPL,mixed-news.com,"[{'id': '806457af1fd9418db78388760f52c06f', 'n...","[{'data': 'Apple', 'type': 'ORG', 'mentions': ...",0.038229,0.064935,0.896836,0.000000,...,86709100.0,96389800.0,1,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Monday
1,2022-01-03,1,AAPL,mixed-news.com,"[{'id': '5985872d662c4a3f941a9c581a9fce71', 'n...","[{'data': 'Apple', 'type': 'ORG', 'mentions': ...",0.048013,0.028628,0.923359,0.000000,...,86709100.0,96389800.0,1,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Monday
2,2022-01-03,1,AAPL,ft.com,"[{'id': '5985872d662c4a3f941a9c581a9fce71', 'n...","[{'data': 'Apple', 'type': 'ORG', 'mentions': ...",0.548833,0.371926,0.079241,0.068182,...,86709100.0,96389800.0,0,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Monday
3,2022-01-04,2,AAPL,ft.com,"[{'id': 'aff4495e1d3d40278d6bb5f9d2a3bd25', 'n...","[{'data': 'US', 'type': 'GPE', 'mentions': 4},...",0.050658,0.097955,0.851387,0.000000,...,86709100.0,96389800.0,0,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Tuesday
4,2022-01-04,3,AAPL,ft.com,"[{'id': 'f1e3e2243cd142408136bd8f43282c4a', 'n...","[{'data': 'City', 'type': 'ORG', 'mentions': 2...",0.065301,0.769178,0.165521,0.250000,...,86709100.0,96389800.0,0,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17466,2023-12-27,1,XOM,marketbeat.com,"[{'id': '52b581be4bea4c7ca107c0261d5d942e', 'n...","[{'data': 'Exxon Mobil Co.', 'type': 'ORG', 'm...",0.536791,0.039401,0.423808,0.000000,...,42628800.0,38421425.0,1,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Wednesday
17467,2023-12-27,13,XOM,other,"[{'id': '4de51cf8472b4ce7a5aecdf52a1de4c0', 'n...","[{'data': 'US', 'type': 'GPE', 'mentions': 8},...",0.740931,0.040317,0.218752,0.000000,...,42628800.0,38421425.0,0,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Wednesday
17468,2023-12-27,8,XOM,other,"[{'id': '2fc8215b27ff48ec9227ea6ae39c5054', 'n...","[{'data': 'U.S.', 'type': 'GPE', 'mentions': 8...",0.188013,0.415904,0.396084,0.000000,...,42628800.0,38421425.0,0,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Wednesday
17469,2023-12-28,1,XOM,defenseworld.net,"[{'id': '52b581be4bea4c7ca107c0261d5d942e', 'n...","[{'data': 'Exeter Financial LLC', 'type': 'ORG...",0.484463,0.045062,0.470475,0.000000,...,42628800.0,38421425.0,1,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Thursday


articleDate           datetime64[ns]
n_companies                    int64
StockSymbol                   object
source                        object
companies                     object
entities                      object
positive_sentiment           float64
negative_sentiment           float64
neutral_sentiment            float64
title_polarity               float64
title_subjectivity           float64
open                         float64
close                        float64
high                         float64
low                          float64
volume_start                 float64
volume_end                   float64
volume_mean                  float64
country_isUS                   int64
open_marketcap               float64
close_marketcap              float64
weekly_change                float64
up_down                       object
percent_change               float64
invest                        object
day_of_week                   object
dtype: object

In [119]:
X_train, X_test, y_train, y_test = train_test_split(df, df.up_down, test_size = .5)

lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open", "volume_start"]
lstCat = ["StockSymbol", "source", "day_of_week"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeClassifier() )
], verbose=True)

pipe.fit( X_train, y_train )
predTrain = pipe.predict( X_train )
predTest = pipe.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrain )
sklearn.metrics.balanced_accuracy_score( y_test, predTest )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.1s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', DecisionTreeClassifier())],
         verbose=True)

1.0

0.9990608033004147

In [120]:
print(df['source'].value_counts())

source
other                           11175
ft.com                           1081
defenseworld.net                  774
finance.yahoo.com                 480
nasdaq.com                        476
benzinga.com                      291
fool.com                          250
etfdailynews.com                  236
reporter.am                       201
marketbeat.com                    173
marketscreener.com                156
mixed-news.com                    150
seekingalpha.com                  149
openpr.com                        141
ca.sports.yahoo.com               127
xrtoday.com                       119
dontworrybuy.com                  106
marketwatch.com                   101
digitaljournal.com                 97
devdiscourse.com                   88
beststocks.com                     88
news.blogtop10.com                 80
cnbc.com                           79
ca.finance.yahoo.com               72
gearrice.com                       72
investorsobserver.com              70
econo

In [121]:
from sklearn.ensemble import RandomForestClassifier
pipeRf = Pipeline([
('preprocessor', preprocessor ),
('model', RandomForestClassifier() )
], verbose=True )

pipeRf.fit( X_train, y_train )
predTrainRf = pipeRf.predict( X_train )
predTestRf = pipeRf.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrainRf )
sklearn.metrics.balanced_accuracy_score( y_test, predTestRf )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   1.8s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', RandomForestClassifier())],
         verbose=True)

1.0

0.9624161002094822

In [122]:
from sklearn.ensemble import GradientBoostingClassifier
pipeGbA = Pipeline([
('preprocessor', preprocessor ),
('model', GradientBoostingClassifier() )
], verbose=True )

pipeGbA.fit( X_train, y_train )
predTrainGbA = pipeGbA.predict( X_train )
predTestGbA = pipeGbA.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrainGbA )
sklearn.metrics.balanced_accuracy_score( y_test, predTestGbA )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   2.5s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', GradientBoostingClassifier())],
         verbose=True)

0.9844205942256663

0.9813483284169124

In [123]:
from sklearn.neural_network import MLPClassifier 

pipeNn555 = Pipeline([
('preprocessor', preprocessor ),
('model', MLPClassifier( hidden_layer_sizes=(5,5,5) ) )
], verbose=True )

pipeNn555.fit( X_train, y_train )
predTrainNn555 = pipeNn555.predict( X_train )
predTestNn555 = pipeNn555.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrainNn555 )
sklearn.metrics.balanced_accuracy_score( y_test, predTestNn555 )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   4.8s


C:\Users\18323\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', MLPClassifier(hidden_layer_sizes=(5, 5, 5)))],
         verbose=True)

0.7701587064287445

0.7436701081612586

In [124]:
pipeNn100 = Pipeline([
('preprocessor', preprocessor ),
('model', MLPClassifier( hidden_layer_sizes=(100, 100, 100) ) )
], verbose=True )

pipeNn100.fit( X_train, y_train )
predTrainNn100 = pipeNn100.predict( X_train )
predTestNn100 = pipeNn100.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrainNn100 )
sklearn.metrics.balanced_accuracy_score( y_test, predTestNn100 )



[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   7.6s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', MLPClassifier(hidden_layer_sizes=(100, 100, 100)))],
         verbose=True)

0.9968880916609867

0.8586582446240008

In [125]:
#Regression
X_train, X_test, y_train, y_test = train_test_split(df, df.percent_change, test_size = .5)

lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open", "volume_start"]
lstCat = ["StockSymbol", "source"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

from sklearn.linear_model import LinearRegression

pipeRg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())  # Linear regression model
])

pipeRg.fit( X_train, y_train )
predTrainRg = pipeRg.predict( X_train )
predTestRg = pipeRg.predict( X_test )

sklearn.metrics.r2_score( y_train, predTrainRg )
sklearn.metrics.r2_score( y_test, predTestRg )

Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('regressor', LinearRegression())])

0.0624236325868035

0.05726517880260318

In [114]:
#Regression
X_train, X_test, y_train, y_test = train_test_split(df, df.percent_change, test_size = .5)

lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open", "volume_start"]
lstCat = ["StockSymbol", "source", "day_of_week"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

from sklearn.neural_network import MLPRegressor

pipeRg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000))
])

pipeRg.fit( X_train, y_train )
predTrainRg = pipeRg.predict( X_train )
predTestRg = pipeRg.predict( X_test )

sklearn.metrics.r2_score( y_train, predTrainRg )
sklearn.metrics.r2_score( y_test, predTestRg )


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('regressor',
                 MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000))])

0.9576686062077415

0.5925084569033725

In [133]:
df['Predicted'] = pipeRg.predict(df)
df
df.isna().sum()

,articleDate,n_companies,StockSymbol,source,companies,entities,positive_sentiment,negative_sentiment,neutral_sentiment,title_polarity,...,volume_mean,country_isUS,open_marketcap,close_marketcap,weekly_change,up_down,percent_change,invest,day_of_week,Predicted
0,2022-01-03,2,AAPL,mixed-news.com,"[{'id': '806457af1fd9418db78388760f52c06f', 'n...","[{'data': 'Apple', 'type': 'ORG', 'mentions': ...",0.038229,0.064935,0.896836,0.000000,...,96389800.0,1,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Monday,-2.093443
1,2022-01-03,1,AAPL,mixed-news.com,"[{'id': '5985872d662c4a3f941a9c581a9fce71', 'n...","[{'data': 'Apple', 'type': 'ORG', 'mentions': ...",0.048013,0.028628,0.923359,0.000000,...,96389800.0,1,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Monday,-2.094587
2,2022-01-03,1,AAPL,ft.com,"[{'id': '5985872d662c4a3f941a9c581a9fce71', 'n...","[{'data': 'Apple', 'type': 'ORG', 'mentions': ...",0.548833,0.371926,0.079241,0.068182,...,96389800.0,0,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Monday,-0.588528
3,2022-01-04,2,AAPL,ft.com,"[{'id': 'aff4495e1d3d40278d6bb5f9d2a3bd25', 'n...","[{'data': 'US', 'type': 'GPE', 'mentions': 4},...",0.050658,0.097955,0.851387,0.000000,...,96389800.0,0,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Tuesday,-0.907580
4,2022-01-04,3,AAPL,ft.com,"[{'id': 'f1e3e2243cd142408136bd8f43282c4a', 'n...","[{'data': 'City', 'type': 'ORG', 'mentions': 2...",0.065301,0.769178,0.165521,0.250000,...,96389800.0,0,1.858108e+10,1.492871e+10,5.660004,up,-3.182817,no invest,Tuesday,-0.918692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17466,2023-12-27,1,XOM,marketbeat.com,"[{'id': '52b581be4bea4c7ca107c0261d5d942e', 'n...","[{'data': 'Exxon Mobil Co.', 'type': 'ORG', 'm...",0.536791,0.039401,0.423808,0.000000,...,38421425.0,1,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Wednesday,0.698423
17467,2023-12-27,13,XOM,other,"[{'id': '4de51cf8472b4ce7a5aecdf52a1de4c0', 'n...","[{'data': 'US', 'type': 'GPE', 'mentions': 8},...",0.740931,0.040317,0.218752,0.000000,...,38421425.0,0,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Wednesday,0.744634
17468,2023-12-27,8,XOM,other,"[{'id': '2fc8215b27ff48ec9227ea6ae39c5054', 'n...","[{'data': 'U.S.', 'type': 'GPE', 'mentions': 8...",0.188013,0.415904,0.396084,0.000000,...,38421425.0,0,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Wednesday,0.289087
17469,2023-12-28,1,XOM,defenseworld.net,"[{'id': '52b581be4bea4c7ca107c0261d5d942e', 'n...","[{'data': 'Exeter Financial LLC', 'type': 'ORG...",0.484463,0.045062,0.470475,0.000000,...,38421425.0,1,5.599066e+09,8.207323e+09,1.080002,up,-0.557823,no invest,Thursday,0.879135


articleDate           0
n_companies           0
StockSymbol           0
source                0
companies             0
entities              0
positive_sentiment    0
negative_sentiment    0
neutral_sentiment     0
title_polarity        0
title_subjectivity    0
open                  0
close                 0
high                  0
low                   0
volume_start          0
volume_end            0
volume_mean           0
country_isUS          0
open_marketcap        0
close_marketcap       0
weekly_change         0
up_down               0
percent_change        0
invest                0
day_of_week           0
Predicted             0
dtype: int64

In [134]:
X_train, X_test, y_train, y_test = train_test_split(df, df.up_down, test_size = .5)
lstNum = ["n_companies", "positive_sentiment", "negative_sentiment", "neutral_sentiment", "title_polarity", "title_subjectivity", "open", "volume_start", "Predicted"]
lstCat = ["StockSymbol", "source", "day_of_week"]

pipeNum = Pipeline( [
('selector', ColumnTransformer([ ('selector', 'passthrough', lstNum) ] ) ),
('scaler', StandardScaler() )
])
pipeCat = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False ) )
])
preprocessor = FeatureUnion([
('cat', pipeCat ),
('num', pipeNum )
])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeClassifier() )
], verbose=True)


In [135]:

pipe.fit( X_train, y_train )
predTrain = pipe.predict( X_train )
predTest = pipe.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrain )
sklearn.metrics.balanced_accuracy_score( y_test, predTest )



[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.1s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start',
                                                                                                    'Predicted'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', DecisionTreeClassifier())],
         verbose=True)

1.0

0.9988753140415684

In [136]:
pipeNn100 = Pipeline([
('preprocessor', preprocessor ),
('model', MLPClassifier( hidden_layer_sizes = (100, 100, 100) ) )
], verbose=True )

pipeNn100.fit( X_train, y_train )
predTrainNn100 = pipeNn100.predict( X_train )
predTestNn100 = pipeNn100.predict( X_test )

from sklearn.metrics import r2_score

r2 = r2_score(y_train, predTrainNn100)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   8.8s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start',
                                                                                                    'Predicted'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', MLPClassifier(hidden_layer_sizes=(100, 100, 100)))],
         verbose=True)

ValueError: could not convert string to float: 'up'

In [137]:
from sklearn.ensemble import GradientBoostingClassifier
pipeGbA = Pipeline([
('preprocessor', preprocessor ),
('model', GradientBoostingClassifier() )
], verbose=True )

pipeGbA.fit( X_train, y_train )
predTrainGbA = pipeGbA.predict( X_train )
predTestGbA = pipeGbA.predict( X_test )

sklearn.metrics.balanced_accuracy_score( y_train, predTrainGbA )
sklearn.metrics.balanced_accuracy_score( y_test, predTestGbA )

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ............. (step 2 of 2) Processing model, total=   2.9s


Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('cat',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['StockSymbol',
                                                                                                    'source',
                                                                                                    'day_of_week'])])),
                                                                 ('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                dtype=<class 'int'>,
                                                                                sparse_output=False))])),
                                                ('num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnTransformer(transformers=[('selector',
                                                                                                   'passthrough',
                                                                                                   ['n_companies',
                                                                                                    'positive_sentiment',
                                                                                                    'negative_sentiment',
                                                                                                    'neutral_sentiment',
                                                                                                    'title_polarity',
                                                                                                    'title_subjectivity',
                                                                                                    'open',
                                                                                                    'volume_start',
                                                                                                    'Predicted'])])),
                                                                 ('scaler',
                                                                  StandardScaler())]))])),
                ('model', GradientBoostingClassifier())],
         verbose=True)

0.9894327287230582

0.9901480852761269

In [ ]:
pipeGbA = Pipeline([
('preprocessor', preprocessor ),
('model', GradientBoostingClassifier() )
], verbose=True )

pipeNn = Pipeline([
('preprocessor', preprocessor ),
('model', MLPClassifier( max_iter=5000 ) )
], verbose=True 
    
 pipeLogistic = Pipeline([
('selector', ColumnTransformer([ ('selector', 'passthrough', lstCat ) ] )),
('encoder', OneHotEncoder( dtype=int, drop='first', sparse_output=False, handle_unknown='ignore'
) ),
('model', LogisticRegression() )
])

paramGridGbA = {'model__learning_rate': [ 0, 0.15, 0.3, 0.45, 0.6, 0.75 ],
'model__max_depth': [1, 3, 6, 9]
}
scorerPaidPrecision = sklearn.metrics.make_scorer( sklearn.metrics.precision_score, pos_label="invest")
gridNnPaidPrecision = GridSearchCV( pipeGbA, paramGridGbA, scoring=scorerPaidPrecision, cv=5, n_jobs=-1
)
gridNnPaidPrecision.fit( X_train, y_train )
gridNnPaidPrecision.best_params_

In [113]:
 pipeNn = Pipeline([
('preprocessor', preprocessor ),
('model', MLPClassifier( max_iter=5000 ) )
], verbose=True )

paramGridNn = {'model__hidden_layer_sizes': [ (5), (10), (100), (10,10), (10,10,10), (50,50,50) ],
'model__activation': ['identity', 'logistic', 'tanh', 'relu']
}

scorerPaidPrecision = sklearn.metrics.make_scorer( sklearn.metrics.precision_score, pos_label="Paid")
gridNnPaidPrecision = GridSearchCV( pipeNn, paramGridNn, scoring=scorerPaidPrecision, cv=5, n_jobs=-1
)
gridNnPaidPrecision.fit( X_train, y_train )
gridNnPaidPrecision.best_params_
gridNnPaidPrecision.best_score

C:\Users\18323\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.8s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        FeatureUnion(transformer_list=[('cat',
                                                                        Pipeline(steps=[('selector',
                                                                                         ColumnTransformer(transformers=[('selector',
                                                                                                                          'passthrough',
                                                                                                                          ['StockSymbol',
                                                                                                                           'source',
                                                                                                                           'day_of_week'])])),
                                                                                        ('encoder',
                                                                                         OneHotEncoder(drop='first',
                                                                                                       dtype=<class 'int'>,
                                                                                                       sparse_output=False))])),
                                                                       ('num',
                                                                        Pipeline(steps=[('selector',
                                                                                         Column...
                                                                                                                           'title_subjectivity',
                                                                                                                           'open',
                                                                                                                           'volume_start',
                                                                                                                           'Predicted'])])),
                                                                                        ('scaler',
                                                                                         StandardScaler())]))])),
                                       ('model', MLPClassifier(max_iter=5000))],
                                verbose=True),
             n_jobs=-1,
             param_grid={'model__activation': ['identity', 'logistic', 'tanh',
                                               'relu'],
                         'model__hidden_layer_sizes': [5, 10, 100, (10, 10),
                                                       (10, 10, 10),
                                                       (50, 50, 50)]},
             scoring=make_scorer(precision_score, pos_label=Paid))

{'model__activation': 'identity', 'model__hidden_layer_sizes': 5}

AttributeError: 'GridSearchCV' object has no attribute 'best_score'